In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import tensorflow as tf
tf.VERSION

'1.15.0'

In [0]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
import numpy as np
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

optimizer = Adam(0.0002, 0.5)


def Generator():
  model = Sequential()
  model.add(Dense(256, input_dim = latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()
  noise = Input(shape = (latent_dim, ))
  img = model(noise)
  return Model(inputs = noise, outputs = img)

def Discriminator():
  model = Sequential()
  model.add(Flatten(input_shape = img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.summary()

  img = Input(shape = img_shape)
  validity = model(img)

  return Model(inputs = img, outputs = validity)

# Build and compile the discriminator
D = Discriminator()
D.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Build the generator
G = Generator()

# The generator takes noise as input and generates imgs
z = Input(shape = (latent_dim, ))
img = G(z)

# For the combined model we will only train the generator
D.trainable = False

# The discriminator takes generated images as input and determines validity
validity = D(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(inputs= z, outputs = validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

In [0]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import os

os.chdir('/content/drive/My Drive/GitHub Repositories')
baseDir = './Generative Adversarial Network'
outputDir = os.path.join(baseDir, 'outputs')
if not os.path.exists(outputDir):
  os.makedirs(outputDir)
  print('Output Directory Created to save Results')

epochs=10000
batch_size=64
save_interval=50

# Load the dataset
(X_train, _), (_, _) = mnist.load_data()
# Rescale -1 to 1
X_train = X_train / 127.5 - 1.
X_train = np.expand_dims(X_train, axis=3)

# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = G.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(outputDir+'/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
    plt.close()


for epoch in range(epochs):
  epoch = epoch + 1

  # ---------------------
  #  Train Discriminator
  # ---------------------

  # Select a random half of images
  idx = np.random.randint(0, X_train.shape[0], batch_size)
  imgs = X_train[idx]

  # Sample noise and generate a batch of new images
  noise = np.random.normal(0, 1, (batch_size, latent_dim))
  gen_imgs = G.predict(noise)

  # Train the discriminator (real classified as ones and generated as zeros)
  d_loss_real = D.train_on_batch(imgs, valid)
  d_loss_fake = D.train_on_batch(gen_imgs, fake)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # ---------------------
  #  Train Generator
  # ---------------------

  # Train the generator (wants discriminator to mistake images as real)
  g_loss = combined.train_on_batch(noise, valid)

  # Plot the progress
  print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

  # If at save interval => save generated image samples
  if epoch % save_interval == 0:
      save_imgs(epoch)






Output hidden; open in https://colab.research.google.com to view.

In [0]:
import imageio
import glob
images = []
for file_name in os.listdir(outputDir):
    if file_name.endswith('.png'):
        file_path = os.path.join(outputDir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave(os.path.join(baseDir, 'GAN-training-outputs.gif'), images, fps = 3)